In [1]:
"""
Define our CalcOpr function and the Match class that we use to interface with it
"""

from collections import namedtuple
import numpy.matlib as mat
import numpy as np
from numpy.linalg import inv

Match = namedtuple("Match", ["teams", "score"])

def CalcOpr(matches):
    """
    Given a list of matches, to least squares OPR calculation
    """
    num_matches = len(matches) # rows
    teams = list(set().union(*[match.teams for match in matches]))
    num_teams = len(teams)
    
    alliances = mat.zeros((num_matches, num_teams))
    scores = mat.zeros((num_matches, 1))
    
    for idx, match in enumerate(matches):
        scores[idx, 0] = match.score
        for team in match.teams:
            alliances[idx, teams.index(team)] = 1
    
    least_squares_approx = np.dot(
        inv(np.dot(np.transpose(alliances), alliances)),
        np.dot(np.transpose(alliances), scores))
    
    oprs = {}
    for idx, team in enumerate(teams):
        oprs[team] = least_squares_approx[idx, 0] 
    return oprs

print(CalcOpr([
            Match(teams=['A', 'B'], score=10),
            Match(teams=['A', 'C'], score=13),
            Match(teams=['B', 'C'], score=7),
            Match(teams=['A', 'D'], score=15),
            Match(teams=['B', 'D'], score=10),
        ]))

{'C': 5.0, 'B': 2.2499999999999982, 'A': 7.7500000000000009, 'D': 7.5}


In [2]:
"""
Connect to the blue alliance api.  Also install a cache so that every request
we make is cached to a file for a day.  This doesn't respect the cache headers
and last-changed headers from tba's api, but since we're using old data we can
be pretty sure that the data we're using is fresh.
"""

import requests
import requests_cache

# Cache all api requests in a file called tba_cache.sqlite for a day
# (monkeypatch the requests module that tbapy uses)
requests_cache.install_cache(cache_name="tba_cache", expire_after=60 * 60 * 24)

import tbapy
from pprint import pprint

api_key = "6qOZ9uAEsb4CDrOBNG6ZnIdi9cWBaZ6DHnCSato97Qfo7bBeUwT9NfFt4Gi5sHFN"

tba = tbapy.TBA(api_key)
pprint(tba.status())

{'android': {'latest_app_version': 4020399, 'min_app_version': 4000299},
 'contbuild_enabled': True,
 'current_season': 2018,
 'down_events': [],
 'ios': {'latest_app_version': -1, 'min_app_version': -1},
 'is_datafeed_down': False,
 'json': {'android': {'latest_app_version': 4020399,
                      'min_app_version': 4000299},
          'contbuild_enabled': True,
          'current_season': 2018,
          'down_events': [],
          'ios': {'latest_app_version': -1, 'min_app_version': -1},
          'is_datafeed_down': False,
          'max_season': 2018,
          'web': {'commit_time': '2018-02-09 14:37:30 -0500',
                  'current_commit': '3dd2c80b8ec53047bdef65b60929b6293b9e98a6',
                  'deploy_time': 'Fri Feb  9 19:51:54 UTC 2018',
                  'travis_job': '339614111'}},
 'max_season': 2018,
 'web': {'commit_time': '2018-02-09 14:37:30 -0500',
         'current_commit': '3dd2c80b8ec53047bdef65b60929b6293b9e98a6',
         'deploy_time': 'Fri 

In [3]:
"""
Given the matches from an event, let's calculate the OPR for each component.
Current components are raw score, rotor rp, and fuel rp.  
"""


def CalcScoreOpr(matches):
    """
    Takes in Matches (like what tba.event_matches returns) and outputs a dictionary
    mapping team-name to score opr
    """
    match_outcomes = []
    for match in matches:
        match_outcomes.append(
            Match(teams=match['alliances']['blue']['team_keys'],
                  score=match['alliances']['blue']['score'])
        )
        match_outcomes.append(
            Match(teams=match['alliances']['red']['team_keys'],
                  score=match['alliances']['red']['score'])
        )
    return CalcOpr(match_outcomes)


def CalcRotorRpOpr(matches):
    """
    Calculate each team's opr with respect to getting a robot bonus.  Rotor bonus is
    pretty nonlinear so part of the experiment here is seeing how well we can predict
    an alliance's liklihood of getting the rotor RP
    """
    match_outcomes = []
    for match in matches:
        if match['comp_level'] == 'qm':
            match_outcomes.append(
                Match(teams=match['alliances']['blue']['team_keys'],
                      score=match['score_breakdown']['blue']['rotorRankingPointAchieved'])
            )
            match_outcomes.append(
                Match(teams=match['alliances']['red']['team_keys'],
                      score=match['score_breakdown']['red']['rotorRankingPointAchieved'])
            )
    return CalcOpr(match_outcomes)
    
    
def CalcFuelRpOpr(matches):
    """
    Calculate each team's opr with respect to getting a fuel bonus.  Fuel bonus
    is pretty nonlinear so part of the experiment here is seeing how well we can predict
    an alliance's likliood of getting the fuel RP
    """
    match_outcomes = []
    for match in matches:
        if match['comp_level'] == 'qm':
            match_outcomes.append(
                Match(teams=match['alliances']['blue']['team_keys'],
                      score=match['score_breakdown']['blue']['kPaRankingPointAchieved'])
            )
            match_outcomes.append(
                Match(teams=match['alliances']['red']['team_keys'],
                      score=match['score_breakdown']['red']['kPaRankingPointAchieved'])
            )
    return CalcOpr(match_outcomes)

In [4]:
"""
Get the OPR statistics about all the teams at a given event.  Cache if possible.  
"""

from collections import namedtuple

EventSummary = namedtuple("EventSummary", ["team_score_opr", "team_rotor_rp_opr", "team_fuel_rp_opr"])

_event_stats = {}

def get_event_stats(event):
    if event not in _event_stats:
        event_matches = tba.event_matches(event)

        _event_stats[event] = EventSummary(
            team_score_opr = CalcScoreOpr(event_matches),
            team_rotor_rp_opr = CalcRotorRpOpr(event_matches),
            team_fuel_rp_opr = CalcFuelRpOpr(event_matches),
        )
    
    return _event_stats[event]
    
pprint(dict(get_event_stats("2017roe")._asdict()))

{'team_fuel_rp_opr': {'frc1002': 0.0013067540141778124,
                      'frc1011': -0.0066918194307005853,
                      'frc115': -0.00027964533450865019,
                      'frc1339': 0.027811071747965696,
                      'frc1414': -0.028274925604767503,
                      'frc1477': 0.030186389459701257,
                      'frc1482': 0.0038428145423401745,
                      'frc1574': 0.81224332275922395,
                      'frc175': 0.0086009350495094769,
                      'frc2183': 0.0017301384649544547,
                      'frc2403': 0.097203821426750783,
                      'frc2468': -0.00038628735476970398,
                      'frc2478': 0.045443848842528646,
                      'frc2485': -0.091623608856357563,
                      'frc2642': 0.019303911285171331,
                      'frc2655': 0.025380401944398757,
                      'frc2881': 0.057278301432307138,
                      'frc2905': -0.020735668480764104

In [5]:
"""
Get the stats for a given team at their most recent region/div-champ.  This includes
their OPR in various categories.  Cache this information if possible.  
"""

TeamSummary = namedtuple("TeamSummary", ["last_event", "score_opr", "rotor_rp_opr", "fuel_rp_opr"])

_team_stats = {}

def get_team_last_event(team, year="2017"):
    events = tba.team_events(team, year)
    events = [
        event
        for event
        in events
        if (event['event_type_string'] in ['Regional', 'District Championship', 'District'] and
            event['key'] != '2017micmp') # 2017micmp is messed up in tba so skip it here
    ]
    
    events = sorted(events, key=lambda x: x['end_date'])
    return events[-1]['event_code']

def get_team_stats(team, year="2017"):
    """
    Get the component opr for the given team based on their most recent regional,
    district champ, or district event
    """
    if team not in _team_stats:
        team_last_event = get_team_last_event(team, year)
        eventSummary = get_event_stats(year + team_last_event)
        
        teamSummary = TeamSummary(
            last_event = team_last_event,
            score_opr = eventSummary.team_score_opr[team],
            rotor_rp_opr = eventSummary.team_rotor_rp_opr[team],
            fuel_rp_opr = eventSummary.team_fuel_rp_opr[team],
        )
        
        _team_stats[team] = teamSummary
    
    return _team_stats[team]

get_team_stats('frc973')

TeamSummary(last_event='cada', score_opr=120.68964289866562, rotor_rp_opr=0.0, fuel_rp_opr=0.44310225380396262)

In [6]:
"""
Simulate one match given the red and blue alliances and creating a MatchOutcome object
"""


import random

MatchOutcome = namedtuple("MatchOutcome", [
        "red_alliance", "blue_alliance",
        "red_score", "blue_score",
        "red_match_rp", "blue_match_rp",
        "red_fuel_rp", "blue_fuel_rp", 
        "red_rotor_rp", "blue_rotor_rp",
        "red_total_rp", "blue_total_rp"
    ])

def simulate_match(red_alliance, blue_alliance):
    red_reports = [get_team_stats(team) for team in red_alliance]
    blue_reports = [get_team_stats(team) for team in blue_alliance]
    
    red_score = sum([
            report.score_opr * random.gauss(1.0, 0.3)
            for report
            in red_reports
    ]) + random.gauss(0.0, 5.0)
    
    blue_score = sum([
            report.score_opr * random.gauss(1.0, 0.3)
            for report
            in blue_reports
    ]) + random.gauss(0.0, 5.0)
    
    if red_score > blue_score:
        red_match_rp = 2
        blue_match_rp = 0
    elif blue_score > red_score:
        blue_match_rp = 2
        red_match_rp = 0
    else:
        blue_match_rp = 1
        red_match_rp = 1
        
    red_rotor_rp_chance = sum([
        report.rotor_rp_opr for report in red_reports
    ])
        
    blue_rotor_rp_chance = sum([
        report.rotor_rp_opr for report in blue_reports
    ])
    
    red_rotor_rp = int(red_rotor_rp_chance > random.random())
    blue_rotor_rp = int(blue_rotor_rp_chance > random.random())
        
    red_fuel_rp_chance = sum([
        report.fuel_rp_opr for report in red_reports
    ])
        
    blue_fuel_rp_chance = sum([
        report.fuel_rp_opr for report in blue_reports
    ])
    
    red_fuel_rp = int(red_fuel_rp_chance > random.random())
    blue_fuel_rp = int(blue_fuel_rp_chance > random.random())
    
    return MatchOutcome(
        red_alliance = red_alliance, blue_alliance = blue_alliance,
        red_score = red_score, blue_score = blue_score,
        red_match_rp = red_match_rp, blue_match_rp = blue_match_rp,
        red_fuel_rp = red_fuel_rp, blue_fuel_rp = blue_fuel_rp,
        red_rotor_rp = red_rotor_rp, blue_rotor_rp = blue_rotor_rp,
        red_total_rp = red_match_rp + red_fuel_rp + red_rotor_rp,
        blue_total_rp = blue_match_rp + blue_fuel_rp + blue_rotor_rp,
    )

for _ in range(3):
    pprint(simulate_match(["frc973", "frc1011", "frc492"], ["frc254", "frc1678", "frc294"])._asdict())

OrderedDict([('red_alliance', ['frc973', 'frc1011', 'frc492']),
             ('blue_alliance', ['frc254', 'frc1678', 'frc294']),
             ('red_score', 314.96167046457987),
             ('blue_score', 464.20365658137854),
             ('red_match_rp', 0),
             ('blue_match_rp', 2),
             ('red_fuel_rp', 1),
             ('blue_fuel_rp', 1),
             ('red_rotor_rp', 0),
             ('blue_rotor_rp', 1),
             ('red_total_rp', 1),
             ('blue_total_rp', 4)])
OrderedDict([('red_alliance', ['frc973', 'frc1011', 'frc492']),
             ('blue_alliance', ['frc254', 'frc1678', 'frc294']),
             ('red_score', 308.22419202894878),
             ('blue_score', 512.60269756742389),
             ('red_match_rp', 0),
             ('blue_match_rp', 2),
             ('red_fuel_rp', 0),
             ('blue_fuel_rp', 1),
             ('red_rotor_rp', 0),
             ('blue_rotor_rp', 0),
             ('red_total_rp', 0),
             ('blue_total_rp', 3)]

In [7]:
"""
Given a list of scheduled matches, simulate the entire event keeping track of
how many ranking points each team gets and returning the rankings at the end
of quals.  
"""

def get_schedule(event):
    """
    Given an event key return the schedule (a list of (red_alliance, blue_alliance) tuples)
    """
    schedule = []
    event_matches = tba.event_matches(event)
    for match in event_matches:
        schedule.append((match['alliances']['red']['team_keys'],
                         match['alliances']['blue']['team_keys']))
    
    return schedule

def simulate_schedule(schedule):
    """
    Given a schedule, simulate the outcomes of all matches
    """
    outcomes = []
    for match in schedule:
        outcomes.append(simulate_match(match[0], match[1]))
    return outcomes

def get_rankings(match_outcomes):
    "Based on the outcomes of each match, determine the rankings"
    teams = set().union(*[match.red_alliance for match in match_outcomes])
    
    team_rps = []
    for team in teams:
        num_rps = 0
        for outcome in match_outcomes:
            if team in outcome.red_alliance:
                num_rps += outcome.red_total_rp
            if team in outcome.blue_alliance:
                num_rps += outcome.blue_total_rp
        team_rps.append((team, num_rps))
    team_rps.sort(key=lambda x: -x[1])
    
    rankings = {}
    for idx, (team, rps) in enumerate(team_rps):
        rankings[team] = idx + 1
        
    return rankings
        
def simulate_event(event):
    return get_rankings(simulate_schedule(get_schedule(event)))

pprint(simulate_event("2017roe"))

{'frc1002': 21,
 'frc1011': 27,
 'frc115': 6,
 'frc1339': 53,
 'frc1414': 31,
 'frc1477': 32,
 'frc1482': 41,
 'frc1574': 1,
 'frc175': 48,
 'frc2183': 47,
 'frc2403': 36,
 'frc2468': 10,
 'frc2478': 28,
 'frc2485': 65,
 'frc2642': 14,
 'frc2655': 60,
 'frc2881': 43,
 'frc2905': 42,
 'frc2928': 5,
 'frc3140': 19,
 'frc3158': 49,
 'frc3229': 17,
 'frc3316': 33,
 'frc3402': 54,
 'frc365': 2,
 'frc3653': 25,
 'frc3824': 11,
 'frc3826': 22,
 'frc3834': 64,
 'frc3991': 50,
 'frc4060': 44,
 'frc418': 8,
 'frc4191': 66,
 'frc4219': 45,
 'frc4265': 9,
 'frc4276': 63,
 'frc435': 13,
 'frc4371': 61,
 'frc441': 37,
 'frc4561': 12,
 'frc4590': 34,
 'frc4592': 20,
 'frc4723': 18,
 'frc488': 59,
 'frc5026': 35,
 'frc5472': 30,
 'frc5499': 15,
 'frc5515': 39,
 'frc5614': 56,
 'frc5803': 29,
 'frc5816': 40,
 'frc585': 55,
 'frc5970': 26,
 'frc6144': 51,
 'frc624': 3,
 'frc6304': 58,
 'frc6325': 46,
 'frc6361': 57,
 'frc6388': 52,
 'frc6409': 62,
 'frc6508': 24,
 'frc6560': 38,
 'frc6705': 16,
 'frc8':

In [8]:
"""
Simulate the event many times monte carlo and keep track of each team's
rankings each time.
"""

AggregateRanking = namedtuple("AggregateRanking", ["p0", "p100", "p50", "p75", "p25", "std_dev"])

def monte_carlo_event(event, iterations=10000):
    team_ranks = {}
    
    for _ in range(iterations):
        rankings = simulate_event(event)
        for team, rank in rankings.items():
            if team not in team_ranks:
                team_ranks[team] = []
            team_ranks[team].append(rank)
    
    aggregate_rankings = {}
    for team, rankings in team_ranks.items():
        aggregate_rankings[team] = AggregateRanking(
            p0=np.percentile(rankings, 0),
            p100=np.percentile(rankings, 100),
            p50=np.percentile(rankings, 50),
            p75=np.percentile(rankings, 75),
            p25=np.percentile(rankings, 25),
            std_dev=np.std(rankings),
        )
    
    return aggregate_rankings

def get_actual_ranking(event, team):
    event_rankings = tba.event_rankings(event)
    for ranking in event_rankings['rankings']:
        if ranking['team_key'] == team:
            return ranking['rank']
    raise Exception("Team Not Found: " + team)

def display_monte_carlo_result(monte_carlo_result, event):
    monte_carlo_result = list(monte_carlo_result.items())
    monte_carlo_result.sort(key=lambda x: (x[1].p50, x[1].p75))
    
    for team, agr_rank in monte_carlo_result:
        print("Predicted Rank {0:2.0f}: {1:10s} (25p={2:2.0f}, 75p={3:2.0f})   Actual Rank: {4}".format(
                agr_rank.p50, team, agr_rank.p25, agr_rank.p75, get_actual_ranking(event, team)))

display_monte_carlo_result(monte_carlo_event("2017roe"), "2017roe")

Predicted Rank  2: frc1574    (25p= 1, 75p= 3)   Actual Rank: 4
Predicted Rank  2: frc973     (25p= 1, 75p= 4)   Actual Rank: 1
Predicted Rank  4: frc365     (25p= 2, 75p= 7)   Actual Rank: 7
Predicted Rank  6: frc2928    (25p= 4, 75p= 9)   Actual Rank: 38
Predicted Rank  6: frc1011    (25p= 4, 75p=10)   Actual Rank: 8
Predicted Rank  8: frc4561    (25p= 5, 75p=12)   Actual Rank: 29
Predicted Rank 10: frc418     (25p= 7, 75p=15)   Actual Rank: 42
Predicted Rank 10: frc2468    (25p= 6, 75p=16)   Actual Rank: 37
Predicted Rank 10: frc115     (25p= 6, 75p=16)   Actual Rank: 2
Predicted Rank 11: frc8       (25p= 7, 75p=17)   Actual Rank: 36
Predicted Rank 12: frc4265    (25p= 8, 75p=18)   Actual Rank: 5
Predicted Rank 12: frc1002    (25p= 8, 75p=19)   Actual Rank: 14
Predicted Rank 12: frc6705    (25p= 8, 75p=19)   Actual Rank: 3
Predicted Rank 13: frc624     (25p= 8, 75p=18)   Actual Rank: 28
Predicted Rank 14: frc4592    (25p=10, 75p=20)   Actual Rank: 6
Predicted Rank 15: frc3824    (25

In [9]:
display_monte_carlo_result(monte_carlo_event("2017new"), "2017new")

Predicted Rank  1: frc118     (25p= 1, 75p= 1)   Actual Rank: 1
Predicted Rank  2: frc1678    (25p= 2, 75p= 3)   Actual Rank: 3
Predicted Rank  3: frc330     (25p= 2, 75p= 4)   Actual Rank: 2
Predicted Rank  4: frc4188    (25p= 3, 75p= 4)   Actual Rank: 18
Predicted Rank  5: frc180     (25p= 4, 75p= 6)   Actual Rank: 7
Predicted Rank  6: frc3663    (25p= 6, 75p= 7)   Actual Rank: 29
Predicted Rank  9: frc3008    (25p= 7, 75p=13)   Actual Rank: 4
Predicted Rank 10: frc4486    (25p= 8, 75p=13)   Actual Rank: 5
Predicted Rank 10: frc3255    (25p= 8, 75p=14)   Actual Rank: 12
Predicted Rank 11: frc3647    (25p= 9, 75p=15)   Actual Rank: 8
Predicted Rank 11: frc3489    (25p= 8, 75p=16)   Actual Rank: 31
Predicted Rank 12: frc696     (25p= 8, 75p=16)   Actual Rank: 59
Predicted Rank 12: frc3473    (25p= 9, 75p=17)   Actual Rank: 9
Predicted Rank 15: frc2073    (25p=11, 75p=20)   Actual Rank: 6
Predicted Rank 16: frc997     (25p=12, 75p=22)   Actual Rank: 17
Predicted Rank 18: frc4469    (25p

In [10]:
display_monte_carlo_result(monte_carlo_event("2017cur"), "2017cur")

Predicted Rank  1: frc2056    (25p= 1, 75p= 1)   Actual Rank: 1
Predicted Rank  2: frc384     (25p= 2, 75p= 2)   Actual Rank: 59
Predicted Rank  3: frc1241    (25p= 3, 75p= 4)   Actual Rank: 12
Predicted Rank  6: frc2481    (25p= 4, 75p= 8)   Actual Rank: 2
Predicted Rank  6: frc203     (25p= 5, 75p= 9)   Actual Rank: 40
Predicted Rank  7: frc4917    (25p= 5, 75p=10)   Actual Rank: 6
Predicted Rank  9: frc6009    (25p= 7, 75p=12)   Actual Rank: 3
Predicted Rank  9: frc2169    (25p= 6, 75p=14)   Actual Rank: 14
Predicted Rank 10: frc230     (25p= 7, 75p=13)   Actual Rank: 11
Predicted Rank 10: frc2791    (25p= 7, 75p=14)   Actual Rank: 45
Predicted Rank 11: frc71      (25p= 8, 75p=16)   Actual Rank: 8
Predicted Rank 12: frc4946    (25p= 8, 75p=18)   Actual Rank: 9
Predicted Rank 15: frc234     (25p=10, 75p=20)   Actual Rank: 44
Predicted Rank 16: frc4039    (25p=12, 75p=22)   Actual Rank: 46
Predicted Rank 17: frc4143    (25p=12, 75p=23)   Actual Rank: 16
Predicted Rank 18: frc1720    (

In [11]:
display_monte_carlo_result(monte_carlo_event("2017gal"), "2017gal")

Predicted Rank  2: frc1538    (25p= 1, 75p= 3)   Actual Rank: 2
Predicted Rank  2: frc5654    (25p= 1, 75p= 4)   Actual Rank: 7
Predicted Rank  3: frc492     (25p= 2, 75p= 5)   Actual Rank: 1
Predicted Rank  4: frc2630    (25p= 2, 75p= 6)   Actual Rank: 46
Predicted Rank  6: frc2576    (25p= 4, 75p= 9)   Actual Rank: 53
Predicted Rank  7: frc3478    (25p= 5, 75p=10)   Actual Rank: 14
Predicted Rank  8: frc3211    (25p= 6, 75p=11)   Actual Rank: 33
Predicted Rank  9: frc2415    (25p= 6, 75p=13)   Actual Rank: 18
Predicted Rank 10: frc498     (25p= 7, 75p=14)   Actual Rank: 3
Predicted Rank 11: frc3635    (25p= 8, 75p=15)   Actual Rank: 32
Predicted Rank 11: frc5074    (25p= 8, 75p=17)   Actual Rank: 17
Predicted Rank 14: frc2231    (25p=10, 75p=21)   Actual Rank: 6
Predicted Rank 15: frc3005    (25p=11, 75p=21)   Actual Rank: 11
Predicted Rank 16: frc587     (25p=12, 75p=21)   Actual Rank: 23
Predicted Rank 19: frc701     (25p=14, 75p=26)   Actual Rank: 13
Predicted Rank 20: frc6579    

In [12]:
display_monte_carlo_result(monte_carlo_event("2017cars"), "2017cars")

Predicted Rank  1: frc195     (25p= 1, 75p= 2)   Actual Rank: 5
Predicted Rank  3: frc125     (25p= 2, 75p= 4)   Actual Rank: 3
Predicted Rank  3: frc5687    (25p= 2, 75p= 4)   Actual Rank: 6
Predicted Rank  4: frc1080    (25p= 3, 75p= 5)   Actual Rank: 26
Predicted Rank  5: frc1796    (25p= 4, 75p= 6)   Actual Rank: 15
Predicted Rank  6: frc33      (25p= 5, 75p= 8)   Actual Rank: 2
Predicted Rank  9: frc1073    (25p= 7, 75p=13)   Actual Rank: 24
Predicted Rank 10: frc5024    (25p= 8, 75p=15)   Actual Rank: 7
Predicted Rank 10: frc2451    (25p= 8, 75p=15)   Actual Rank: 21
Predicted Rank 11: frc177     (25p= 8, 75p=16)   Actual Rank: 27
Predicted Rank 13: frc303     (25p= 9, 75p=18)   Actual Rank: 1
Predicted Rank 13: frc135     (25p=10, 75p=19)   Actual Rank: 4
Predicted Rank 15: frc1018    (25p=11, 75p=22)   Actual Rank: 13
Predicted Rank 15: frc6329    (25p=11, 75p=22)   Actual Rank: 37
Predicted Rank 20: frc876     (25p=15, 75p=28)   Actual Rank: 30
Predicted Rank 20: frc2052    (2

In [13]:
display_monte_carlo_result(monte_carlo_event("2017carv"), "2017carv")

Predicted Rank  2: frc2122    (25p= 1, 75p= 3)   Actual Rank: 3
Predicted Rank  2: frc971     (25p= 1, 75p= 3)   Actual Rank: 2
Predicted Rank  3: frc987     (25p= 2, 75p= 4)   Actual Rank: 5
Predicted Rank  4: frc1690    (25p= 3, 75p= 4)   Actual Rank: 1
Predicted Rank  5: frc3238    (25p= 5, 75p= 6)   Actual Rank: 6
Predicted Rank  6: frc4910    (25p= 5, 75p= 8)   Actual Rank: 33
Predicted Rank  7: frc2974    (25p= 6, 75p= 8)   Actual Rank: 46
Predicted Rank  8: frc2992    (25p= 7, 75p= 9)   Actual Rank: 21
Predicted Rank 10: frc1700    (25p= 9, 75p=12)   Actual Rank: 30
Predicted Rank 11: frc2471    (25p= 9, 75p=13)   Actual Rank: 26
Predicted Rank 12: frc179     (25p=10, 75p=14)   Actual Rank: 14
Predicted Rank 12: frc1937    (25p=10, 75p=14)   Actual Rank: 29
Predicted Rank 12: frc3339    (25p=10, 75p=14)   Actual Rank: 9
Predicted Rank 14: frc3674    (25p=11, 75p=17)   Actual Rank: 23
Predicted Rank 21: frc4334    (25p=16, 75p=29)   Actual Rank: 4
Predicted Rank 21: frc2930    (2

In [14]:
display_monte_carlo_result(monte_carlo_event("2017hop"), "2017hop")

Predicted Rank  1: frc604     (25p= 1, 75p= 2)   Actual Rank: 4
Predicted Rank  2: frc2848    (25p= 2, 75p= 3)   Actual Rank: 8
Predicted Rank  3: frc1619    (25p= 3, 75p= 4)   Actual Rank: 1
Predicted Rank  4: frc1778    (25p= 3, 75p= 6)   Actual Rank: 9
Predicted Rank  7: frc4613    (25p= 5, 75p=10)   Actual Rank: 6
Predicted Rank  7: frc3309    (25p= 5, 75p=10)   Actual Rank: 2
Predicted Rank  8: frc3835    (25p= 6, 75p=12)   Actual Rank: 11
Predicted Rank  9: frc4941    (25p= 7, 75p=12)   Actual Rank: 12
Predicted Rank 11: frc4488    (25p= 8, 75p=14)   Actual Rank: 7
Predicted Rank 11: frc2682    (25p= 8, 75p=15)   Actual Rank: 16
Predicted Rank 11: frc2903    (25p= 8, 75p=16)   Actual Rank: 15
Predicted Rank 12: frc5818    (25p= 9, 75p=15)   Actual Rank: 13
Predicted Rank 13: frc1868    (25p=10, 75p=17)   Actual Rank: 30
Predicted Rank 14: frc5458    (25p=10, 75p=19)   Actual Rank: 10
Predicted Rank 14: frc3314    (25p=10, 75p=19)   Actual Rank: 3
Predicted Rank 15: frc2910    (25

In [15]:
display_monte_carlo_result(monte_carlo_event("2017tes"), "2017tes")

Predicted Rank  1: frc148     (25p= 1, 75p= 2)   Actual Rank: 4
Predicted Rank  3: frc2168    (25p= 2, 75p= 5)   Actual Rank: 3
Predicted Rank  4: frc3464    (25p= 3, 75p= 7)   Actual Rank: 19
Predicted Rank  7: frc469     (25p= 4, 75p=10)   Actual Rank: 1
Predicted Rank  7: frc95      (25p= 4, 75p=11)   Actual Rank: 25
Predicted Rank  8: frc3130    (25p= 4, 75p=12)   Actual Rank: 10
Predicted Rank  9: frc6538    (25p= 6, 75p=13)   Actual Rank: 8
Predicted Rank  9: frc2607    (25p= 6, 75p=13)   Actual Rank: 26
Predicted Rank  9: frc3452    (25p= 6, 75p=13)   Actual Rank: 5
Predicted Rank 10: frc2084    (25p= 6, 75p=14)   Actual Rank: 17
Predicted Rank 10: frc3015    (25p= 7, 75p=14)   Actual Rank: 11
Predicted Rank 11: frc2619    (25p= 7, 75p=15)   Actual Rank: 7
Predicted Rank 11: frc3683    (25p= 7, 75p=16)   Actual Rank: 12
Predicted Rank 14: frc829     (25p=11, 75p=18)   Actual Rank: 2
Predicted Rank 15: frc193     (25p=10, 75p=19)   Actual Rank: 34
Predicted Rank 15: frc1519    (2

In [16]:
"""
Determine how accurate our predictions were.  (in progress)
"""

import math 

def norm_pdf(x, mean, sd):
    """
    Probability distribution function for a normal curve
    """
    var = float(sd) ** 2
    denom = (2 * math.pi * var) ** 0.5
    num = math.exp(-(float(x) - float(mean)) ** 2 / (2 * var))
    return num / denom

def score_monte_carlo_prediction(monte_carlo_result, event):
    """
    Assuming the model we generated using monte carlo metrics, calculate how
    likely the actual rankings were (do sum of log of probabilities because
    of floating point error... closer to zero is higher probability and more
    negative is lesser probability)
    
    Only look at the first 16 or so teams because everything after that will
    be a shitshow regardless.
    """
    monte_carlo_result = list(monte_carlo_result.items())
    monte_carlo_result = sorted(monte_carlo_result, key=lambda x: (x[1].p50, x[1].p75))[:16]
    
    team_probabilities = {}
    for team, prediction in monte_carlo_result:
        mean = prediction.p50
        sd = prediction.std_dev
        #team_probabilities[team] = math.log(norm_pdf(get_actual_ranking(event, team), mean, sd))
        team_probabilities[team] = abs(mean - get_actual_ranking(event, team))
        
    return sum(team_probabilities.values()) / len(team_probabilities)

pprint(score_monte_carlo_prediction(monte_carlo_event("2017roe"), "2017roe"))

12.875
